In [167]:
from config import *
import pandas as pd 


In [168]:
raw_df = pd.read_csv(CFG.data_rootfolder / "person.csv")
df = raw_df
df.head()

/var/folders/lg/ssxqzhrj33g6tvlvnhwsmxxr0000gn/T/ipykernel_28051/139228539.py:1: DtypeWarning: Columns (6,9,10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv(CFG.data_rootfolder / "person.csv")


,ACCIDENT_NO,PERSON_ID,VEHICLE_ID,SEX,AGE,Age Group,INJ_LEVEL,Inj Level Desc,SEATING_POSITION,HELMET_BELT_WORN,ROAD_USER_TYPE,Road User Type Desc,LICENCE_STATE,PEDEST_MOVEMENT,POSTCODE,TAKEN_HOSPITAL,EJECTED_CODE
0,T20060000010,01,A,F,NaN,unknown,4,Not injured,LF,1,3,Passengers,,0,3130.0,,0
1,T20060000010,02,C,M,43.0,40-49,4,Not injured,LF,1,3,Passengers,,0,7310.0,,0
2,T20060000010,03,C,M,22.0,22-25,4,Not injured,LR,1,3,Passengers,,0,7310.0,,0
3,T20060000010,A,A,M,72.0,70+,4,Not injured,D,1,2,Drivers,V,0,3130.0,,0
4,T20060000010,B,B,F,62.0,60-64,3,Other injury,D,1,2,Drivers,V,0,NaN,N,0


In [169]:
#Let's clean the column that will be used as indexes latter on
df["PERSON_ID"] = df["PERSON_ID"].str.replace(" ","")
df["VEHICLE_ID"] = df["VEHICLE_ID"].str.replace(" ","")

In [170]:
drivers_df = df[df["Road User Type Desc"]=="Drivers"]

In [171]:
drivers_df = drivers_df[["ACCIDENT_NO","VEHICLE_ID","AGE","SEX", "LICENCE_STATE","PERSON_ID"]]
drivers_df = drivers_df.rename(columns={"AGE": "Driver Age", "SEX" :"Driver sex", "LICENCE_STATE": " Driver LICENCE_STATE"})
drivers_df.head()

,ACCIDENT_NO,VEHICLE_ID,Driver Age,Driver sex,Driver LICENCE_STATE,PERSON_ID
3,T20060000010,A,72.0,M,V,A
4,T20060000010,B,62.0,F,V,B
5,T20060000010,C,39.0,M,V,C
8,T20060000018,A,30.0,M,V,A
9,T20060000018,B,20.0,F,V,B


In [172]:
drivers_df = drivers_df.set_index(["ACCIDENT_NO","VEHICLE_ID","PERSON_ID" ])

In [173]:
df = df[["ACCIDENT_NO","PERSON_ID","VEHICLE_ID","SEX","AGE","INJ_LEVEL","SEATING_POSITION", "Road User Type Desc"]]
df.head(20)

,ACCIDENT_NO,PERSON_ID,VEHICLE_ID,SEX,AGE,INJ_LEVEL,SEATING_POSITION,Road User Type Desc
0,T20060000010,01,A,F,NaN,4,LF,Passengers
1,T20060000010,02,C,M,43.0,4,LF,Passengers
2,T20060000010,03,C,M,22.0,4,LR,Passengers
3,T20060000010,A,A,M,72.0,4,D,Drivers
4,T20060000010,B,B,F,62.0,3,D,Drivers
5,T20060000010,C,C,M,39.0,4,D,Drivers
6,T20060000018,01,B,F,18.0,4,CR,Passengers
7,T20060000018,02,B,M,18.0,4,RR,Passengers
8,T20060000018,A,A,M,30.0,4,D,Drivers
9,T20060000018,B,B,F,20.0,3,D,Drivers


In [174]:
person_df= pd.merge(df, drivers_df, how="left", on=["ACCIDENT_NO", "VEHICLE_ID"])
all_duplicates = person_df.duplicated(subset=["ACCIDENT_NO","VEHICLE_ID","PERSON_ID"], keep=False)
person_df[all_duplicates]

,ACCIDENT_NO,PERSON_ID,VEHICLE_ID,SEX,AGE,INJ_LEVEL,SEATING_POSITION,Road User Type Desc,Driver Age,Driver sex,Driver LICENCE_STATE
8207,T20060011876,02,A,M,50.0,4,NK,Drivers,50.0,M,V
8208,T20060011876,02,A,M,50.0,4,NK,Drivers,60.0,M,V
8209,T20060011876,A,A,M,60.0,4,NK,Drivers,50.0,M,V
8210,T20060011876,A,A,M,60.0,4,NK,Drivers,60.0,M,V
464602,T20190017221,01,A,U,NaN,4,D,Drivers,NaN,U,
464603,T20190017221,01,A,U,NaN,4,D,Drivers,NaN,U,Z
464604,T20190017221,A,A,U,NaN,4,D,Drivers,NaN,U,
464605,T20190017221,A,A,U,NaN,4,D,Drivers,NaN,U,Z


In [175]:
discarded_duplicates = person_df.duplicated(subset=["ACCIDENT_NO","VEHICLE_ID","PERSON_ID"], keep="first")
person_df= pd.merge(df, drivers_df, how="left", on=["ACCIDENT_NO", "VEHICLE_ID"])

person_df = person_df[~discarded_duplicates].set_index(["ACCIDENT_NO","VEHICLE_ID","PERSON_ID"],verify_integrity=True )
person_df.head(20)

SEX   AGE INJ_LEVEL SEATING_POSITION  \
ACCIDENT_NO  VEHICLE_ID PERSON_ID                                        
T20060000010 A          01          F   NaN         4               LF   
             C          02          M  43.0         4               LF   
                        03          M  22.0         4               LR   
             A          A           M  72.0         4               D    
             B          B           F  62.0         3               D    
             C          C           M  39.0         4               D    
T20060000018 B          01          F  18.0         4               CR   
                        02          M  18.0         4               RR   
             A          A           M  30.0         4               D    
             B          B           F  20.0         3               D    
T20060000022 A          01          F  50.0         2               PL   
                        A           M  45.0         4               D    
T20060000023 A          A           M  43.0         2               D    
             B          B           M  42.0         4               D    
T20060000026 B          01          F  38.0         3               LF   
             A          A           M  84.0         3               D    
             B          B           M  51.0         3               D    
T20060000028 A          01          M  85.0         4               D    
             B          B           M  57.0         2               D    
T20060000035 A          A           F  50.0         2               D    

                                  Road User Type Desc  Driver Age Driver sex  \
ACCIDENT_NO  VEHICLE_ID PERSON_ID                                              
T20060000010 A          01                 Passengers        72.0          M   
             C          02                 Passengers        39.0          M   
                        03                 Passengers        39.0          M   
             A          A                     Drivers        72.0          M   
             B          B                     Drivers        62.0          F   
             C          C                     Drivers        39.0          M   
T20060000018 B          01                 Passengers        20.0          F   
                        02                 Passengers        20.0          F   
             A          A                     Drivers        30.0          M   
             B          B                     Drivers        20.0          F   
T20060000022 A          01         Pillion Passengers         NaN        NaN   
                        A               Motorcyclists         NaN        NaN   
T20060000023 A          A                     Drivers        43.0          M   
             B          B                     Drivers        42.0          M   
T20060000026 B          01                 Passengers        51.0          M   
             A          A                     Drivers        84.0          M   
             B          B                     Drivers        51.0          M   
T20060000028 A          01                    Unknown         NaN        NaN   
             B          B               Motorcyclists         NaN        NaN   
T20060000035 A          A                     Drivers        50.0          F   

                                   Driver LICENCE_STATE  
ACCIDENT_NO  VEHICLE_ID PERSON_ID                        
T20060000010 A          01                            V  
             C          02                            V  
                        03                            V  
             A          A                             V  
             B          B                             V  
             C          C                             V  
T20060000018 B          01                            V  
                        02                            V  
             A          A                             V  
      